In [1]:
import pandas as pd
import numpy as np
import os
import re

from pyspark.sql import SparkSession
from pyspark.shell import spark
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql import functions as F
import matplotlib.pyplot as plt

spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.1.2
      /_/

Using Python version 3.7.4 (default, Aug 13 2019 15:17:50)
Spark context Web UI available at http://172.16.29.12:4042
Spark context available as 'sc' (master = local[*], app id = local-1662578512765).
SparkSession available as 'spark'.


## Loading all datasets
- We will be performing basic analysis on each of the dataset
- Choose certain features that seem to be appropriate for geospatial visualisation

### Customer transaction details

In [102]:
customer_join_transaction = spark.read.parquet("../curated/customer_join_transaction.parquet/")

AnalysisException: Path does not exist: file:/Users/oliver/Documents/GitHub/generic-buy-now-pay-later-project-group-19/curated/customer_join_transaction.parquet

In [ ]:
customer_join_transaction.count()

In [5]:
customer_join_transaction.show()

+-------+------------+------------------+--------------------+--------------+--------+-----+------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|postcode|state|gender|
+-------+------------+------------------+--------------------+--------------+--------+-----+------+
|  14409| 64403598239|116.41150553221357|5474e46e-7073-442...|    2021-08-21|    3874|  VIC|  Male|
|  14409| 45629217853|28.600613410021932|f539493d-175d-48b...|    2021-04-09|    3874|  VIC|  Male|
|  14409| 77505747217|440.68714072131945|99f8e136-0ca2-491...|    2021-08-22|    3874|  VIC|  Male|
|  14409| 68216911708|22.960774428094698|f4ea2269-3509-411...|    2021-04-10|    3874|  VIC|  Male|
|  14409| 24852446429|30.776665837082245|4b1413e3-3ee9-430...|    2021-08-16|    3874|  VIC|  Male|
|  14409| 98072939449| 1118.263275036643|e597537b-d4a9-446...|    2021-04-10|    3874|  VIC|  Male|
|  14409| 22718657980|307.02232595923795|82c8d263-9d95-424...|    2021-08-27|    3874|  VIC|  Male|


### Customer behaviour
- Only notable feature is no_orders of each customer at a particular date

In [6]:
customer_behaviour = spark.read.parquet("../data/curated/customer_purchase_behaviour.parquet/")

In [7]:
customer_behaviour

user_id,order_datetime,dollar_spent,no_orders
18585,2021-08-20,28.66078522665458,2
271,2021-08-20,1851.9709565996084,2
559,2021-08-20,519.619820268801,2
19156,2021-08-20,302.72316980059384,2
756,2021-08-20,169.25922159143067,2
785,2021-08-20,49.4051194612238,1
970,2021-08-20,12.28895749317062,1
997,2021-08-20,57.1290091592229,1
1069,2021-08-20,65.389116401826,1
1256,2021-08-20,337.0946186819374,2


In [3]:
merchant_sales = spark.read.parquet("../data/curated/merchant_sales.parquet/")

### Merchant Sales
- Sales revenue per day
- Number of orders

In [63]:
merchant_sales

merchant_abn,order_datetime,sales_revenue,no_orders,month
51515623602,2021-11-26,4223.72384887157,17,11
68645868338,2021-11-26,6130.220383907341,31,11
71528203369,2021-11-26,7230.3248769568145,137,11
89153112408,2021-11-26,344.70425919137756,4,11
52569909432,2021-11-26,511.04916047476377,13,11
55069630872,2021-11-26,6217.119636236526,21,11
56559128138,2021-11-26,3567.858905697874,13,11
81249812970,2021-11-26,1336.9727113172314,4,11
74780446526,2021-11-26,2787.8913158716205,7,11
78004123157,2021-11-26,30.63249357250481,2,11


In [81]:
merchant_sales = spark.read.parquet("../data/curated/merchant_sales.parquet/")
merchant_sales = merchant_sales.select(merchant_sales['*'], month(col("order_datetime")).alias('month'))
merchant_sales = merchant_sales.select(merchant_sales['*'], year(col("order_datetime")).alias('year'))
merchant_monthly_sales = merchant_sales.groupby(['merchant_abn','year','month']).agg({'sales_revenue':'sum',
                                                                               'no_orders':'sum'})
# merchant_monthly_deviation = merchant_monthly_sales.groupby('merchant_abn').agg(F.mean('sum(sales_revenue)'),
#                                                                                 F.stddev('sum(sales_revenue)'),
#                                                                                 F.mean('sum(no_orders)'),
#                                                                                 F.stddev('sum(no_orders)'))
# coefficient_variation = (merchant_monthly_deviation.withColumn('std/mean',
#                                                                F.col('stddev_samp(sum(sales_revenue))') / F.col('avg(sum(sales_revenue))'))
#                                                    .withColumn('avgsale/avgorder',
#                                                                F.col('avg(sum(sales_revenue))') / F.col('avg(sum(no_orders))'))
#                                                    .toPandas())


In [68]:
merchant_sales.count()

398352

In [84]:
from dateutil import relativedelta
min_date, max_date = merchant_sales.select(min("order_datetime"), max("order_datetime")).first()
r = relativedelta.relativedelta(max_date, min_date)

months_difference = (r.years * 12) + r.months
months_difference

5

In [87]:
num_days = (max_date-min_date).days
num_days

183

In [69]:
merchant_monthly_sales = merchant_sales.groupby(['merchant_abn','month']).agg({'sales_revenue':'sum',
                                                                               'no_orders':'sum'})


In [97]:
merchant_sales.where(F.col('merchant_abn') == 38700038932).groupby('merchant_abn').agg({'sales_revenue':'sum','no_orders':'sum'})

merchant_abn,sum(sales_revenue),sum(no_orders)
38700038932,2129729.112645474,1949


In [98]:
merchant_daily_sales = merchant_sales.groupby('merchant_abn').agg(
    (F.sum(F.col('sales_revenue')) / num_days).alias('avg_daily_rev'),
    (F.sum(F.col('sales_revenue')) / F.sum(F.col('no_orders'))).alias('avg_value_per_order'),
    (F.sum(F.col('no_orders')) / num_days).alias('avg_daily_order')
)

In [107]:
merchant = spark.read.parquet("../data/tables/tbl_merchants.parquet")

In [118]:
merchant_join = merchant.join(merchant_daily_sales, ["merchant_abn"]).toPandas()

In [119]:
merchant_join

,merchant_abn,name,tags,avg_daily_rev,avg_value_per_order,avg_daily_order
0,38700038932,Etiam Bibendum Industries,"[(tent and awning shops), (a), (take rate: 6.31)]",11637.864004,1092.729150,10.650273
1,73256306726,Id LLP,"((health and beauty spas), (b), (take rate: 4....",2558.570139,278.701390,9.180328
2,41956465747,Pede Cras LLP,"[[motor Vehicle supplies and new parts], [b], ...",111.809577,235.185661,0.475410
3,60654402457,Lacus Quisque Imperdiet Institute,"[[digital goods: books, movies, music], [b], [...",23.586067,83.004813,0.284153
4,73841664453,Lacinia At LLP,"([digital goods: books, movies, music], [a], ...",152.210626,89.277386,1.704918
...,...,...,...,...,...,...
3850,81906511933,Sodales Purus In Ltd,"[(bicycle shopS - sales and service), (a), (ta...",54.959161,529.343502,0.103825
3851,87788702467,Massa Suspendisse Industries,"[(health and beauty spas), (b), (take rate: 3....",30.832629,170.980942,0.180328
3852,80821603856,Semper Erat Inc.,"((computers, computer periphEral equipment, an...",23.822817,217.978772,0.109290
3853,26465868807,Volutpat Ornare LLC,"((bicycle shops - sales and sErvice), (b), (ta...",58.270918,1184.841993,0.049180


In [120]:
# Function to extract tags, revenue level and take rate from tags column
def extract_tags(arr, category='tags'):
    
    # Split tags into the three components
    arr = arr[1:-1]
    split_arr = re.split('\), \(|\], \[', arr.strip('[()]'))
    
    if category == 'take_rate':
        return re.findall('[\d\.\d]+', split_arr[2])[0]
    
    elif category == 'revenue_level':
        return split_arr[1].lower()
    
    return split_arr[0].lower()

merchant_join['take_rate'] = merchant_join['tags'].apply(lambda x : extract_tags(x, 'take_rate'))
merchant_join['revenue_level'] = merchant_join['tags'].apply(lambda x : extract_tags(x, 'revenue_level'))
merchant_join['tags'] = merchant_join['tags'].apply(lambda x : extract_tags(x, 'tags'))

In [123]:
merchant_join['take_rate'] = pd.to_numeric(merchant_join['take_rate'])

In [138]:
merchant_join['avg_daily_commission'] = merchant_join['avg_daily_rev'] * (merchant_join['take_rate']/100)
merchant_join['avg_commission_per_order'] = merchant_join['avg_value_per_order'] * (merchant_join['take_rate']/100)

In [140]:
merchant_join.sort_values('avg_commission_per_order', ascending=False)[:20]

,merchant_abn,name,tags,avg_daily_rev,avg_value_per_order,avg_daily_order,take_rate,revenue_level,avg_daily_commision,avg_daily_commission,avg_commission_per_order
561,53877856360,Sem Magna Company,"antique shops - sales, repairs, and restoratio...",11.176637,2045.324480,0.005464,6.43,a,0.718658,0.718658,131.514364
3049,81790520013,Magna Praesent Interdum Incorporated,"equipment, tool, furniture, and appliance rent...",10.539337,1928.698636,0.005464,6.81,a,0.717729,0.717729,131.344377
1271,44345785419,Phasellus Nulla LLC,"jewelry, watch, clock, and silverware shops",11.224958,2054.167327,0.005464,6.07,a,0.681355,0.681355,124.687957
1444,15457638036,Gravida Praesent LLP,telecom,9.772286,1788.328381,0.005464,6.97,a,0.681128,0.681128,124.646488
2896,46054596841,Dolor Sit PC,"equipment, tool, furniture, and appliance rent...",10.854496,1986.372820,0.005464,5.83,a,0.632817,0.632817,115.805535
323,83173175334,Enim Gravida Inc.,"antique shops - sales, repairs, and restoratio...",10.179132,1862.781212,0.005464,5.81,a,0.591408,0.591408,108.227588
164,41027080823,Dolor Donec Ltd,"furniture, home furnishings and equipment shop...",16.669375,1525.247793,0.010929,6.89,a,1.148520,1.148520,105.089573
3115,37358528402,Enim Mauris Inc.,"equipment, tool, furniture, and appliance rent...",9.567690,1750.887280,0.005464,5.99,a,0.573105,0.573105,104.878148
403,97884414539,Ut Corporation,"antique shops - sales, repairs, and restoratio...",16.646503,1523.154987,0.010929,6.82,a,1.135291,1.135291,103.879170
1363,31552582037,Velit Inc.,telecom,11.184389,2046.743273,0.005464,5.04,b,0.563693,0.563693,103.155861


In [83]:
merchant_monthly_sales.withColumn("avg_daily_revenue",
                                   F.col('sum(sales_revenue)')/30)

merchant_abn,year,month,sum(sales_revenue),sum(no_orders),avg_daily_revenue
67988680834,2021,11,11326.69758632274,17,377.556586210758
63978837398,2021,12,73719.99851016694,280,2457.3332836722316
45899477665,2021,12,101981.12214129185,413,3399.370738043062
90796680287,2021,11,35953.715637214606,113,1198.4571879071534
81909596183,2021,10,35173.54241474047,40,1172.4514138246823
40938677569,2021,10,7747.165489582728,118,258.2388496527576
15347709593,2021,10,1500.7299235843022,14,50.02433078614341
25971755977,2021,8,551.9229575560421,4,18.397431918534735
69418675752,2021,9,16797.95959636279,72,559.9319865454264
93077671049,2022,1,8962.424410039894,50,298.7474803346631


In [75]:
monthrange(2021,11)[1]

30

In [72]:
merchant_monthly_sales

merchant_abn,year,month,sum(sales_revenue),sum(no_orders)
67988680834,2021,11,11326.69758632274,17
63978837398,2021,12,73719.99851016694,280
45899477665,2021,12,101981.12214129185,413
90796680287,2021,11,35953.715637214606,113
81909596183,2021,10,35173.54241474047,40
40938677569,2021,10,7747.165489582728,118
15347709593,2021,10,1500.7299235843022,14
25971755977,2021,8,551.9229575560421,4
69418675752,2021,9,16797.95959636279,72
93077671049,2022,1,8962.424410039894,50


In [8]:
merchant_monthly_deviation = merchant_monthly_sales.groupby('merchant_abn').agg(F.mean('sum(sales_revenue)'),
                                                                                F.stddev('sum(sales_revenue)'),
                                                                                F.mean('sum(no_orders)'),
                                                                                F.stddev('sum(no_orders)'))

In [9]:
coefficient_variation = (merchant_monthly_deviation.withColumn('std/mean',
                                                               F.col('stddev_samp(sum(sales_revenue))') / F.col('avg(sum(sales_revenue))'))
                                                   .withColumn('avgsale/avgorder',
                                                               F.col('avg(sum(sales_revenue))') / F.col('avg(sum(no_orders))'))
                                                   .toPandas())

In [14]:
top_10_most_unstable_merchant = coefficient_variation.sort_values(['avg(sum(sales_revenue))'], ascending=False)[:10]

In [15]:
top_10_most_unstable_merchant

,merchant_abn,avg(sum(sales_revenue)),stddev_samp(sum(sales_revenue)),avg(sum(no_orders)),stddev_samp(sum(no_orders)),std/mean,avgsale/avgorder
204,27093785141,446770.003131,220510.254310,1178.142857,576.239947,0.493565,379.215475
807,96680767841,444271.749573,211931.739037,1420.000000,675.054566,0.477032,312.867429
3003,50315283629,441749.657164,210839.619311,1370.285714,656.560917,0.477283,322.377773
1967,32709545238,439766.522559,207116.199488,596.857143,286.004329,0.470969,736.803652
1042,35909341340,438266.701529,210533.642029,1743.714286,832.959926,0.480378,251.340891
1895,86578477987,432645.222931,202777.857303,12317.571429,5786.935022,0.468693,35.124231
1289,28057731482,431425.112851,193553.510710,578.857143,256.745937,0.448638,745.304983
3446,21439773999,429822.010033,202483.270143,5472.428571,2580.848495,0.471086,78.543192
3199,18158387243,429719.719320,206832.112158,760.285714,367.556669,0.481319,565.208199
1255,48534649627,429687.956089,203914.492305,2970.428571,1413.041266,0.474564,144.655206


In [13]:
top_10_most_rev_per_order = coefficient_variation.sort_values('avgsale/avgorder', ascending=False)[:10]
top_10_most_rev_per_order

,merchant_abn,avg(sum(sales_revenue)),stddev_samp(sum(sales_revenue)),avg(sum(no_orders)),stddev_samp(sum(no_orders)),std/mean,avgsale/avgorder
67,38049816588,2075.875900,NaN,1.0,NaN,NaN,2075.875900
3894,66482985683,2072.334889,NaN,1.0,NaN,NaN,2072.334889
2371,71762159356,2069.791694,NaN,1.0,NaN,NaN,2069.791694
1682,65959377833,2055.390127,NaN,1.0,NaN,NaN,2055.390127
1356,44345785419,2054.167327,NaN,1.0,NaN,NaN,2054.167327
527,75265429612,2048.379909,NaN,1.0,NaN,NaN,2048.379909
1480,31552582037,2046.743273,NaN,1.0,NaN,NaN,2046.743273
609,53877856360,2045.324480,NaN,1.0,NaN,NaN,2045.324480
824,57241240228,2034.473530,NaN,1.0,NaN,NaN,2034.473530
1326,79800042168,3036.921031,1470.389244,1.5,0.707107,0.484171,2024.614020


### Sales by region
- Information regarding sales made by customers of specific locations in Australia

In [16]:
sales_by_region = pd.read_parquet("../data/curated/sales_by_region.parquet/")

In [17]:
sales_by_region.head()

,state,postcode,order_datetime,dollar_spent,no_orders
0,NT,841,2021-10-01,1190.514532,4
1,VIC,3123,2021-11-11,2088.477116,13
2,NSW,1360,2021-11-30,1572.944357,10
3,SA,5251,2021-11-18,1176.370464,11
4,VIC,3375,2022-02-04,3149.566721,12


In [18]:
sales_by_region.nunique()

state                  8
postcode            3165
order_datetime       184
dollar_spent      576539
no_orders             53
dtype: int64

In [19]:
sales_by_region

,state,postcode,order_datetime,dollar_spent,no_orders
0,NT,841,2021-10-01,1190.514532,4
1,VIC,3123,2021-11-11,2088.477116,13
2,NSW,1360,2021-11-30,1572.944357,10
3,SA,5251,2021-11-18,1176.370464,11
4,VIC,3375,2022-02-04,3149.566721,12
...,...,...,...,...,...
576584,NSW,2380,2021-08-30,38.844258,1
576585,NSW,1187,2021-08-28,73.372201,3
576586,NSW,2343,2022-01-19,76.887112,2
576587,VIC,3160,2021-09-20,31.002981,1


In [20]:
revenue_by_region = sales_by_region\
                    .groupby(['state', 'postcode'], as_index=False)\
                    .agg(
                        {
                            'dollar_spent': 'sum'
                        }
                    ).rename(columns={'dollar_spent': 'total_revenue_generated'})


In [21]:
revenue_by_region['postcode'] = revenue_by_region['postcode'].astype(int)

In [22]:
revenue_by_region

,state,postcode,total_revenue_generated
0,ACT,200,101102.253779
1,ACT,2600,128671.340198
2,ACT,2601,143713.207764
3,ACT,2602,165889.186214
4,ACT,2603,163251.853817
...,...,...,...
3160,WA,6989,249449.669191
3161,WA,6990,169500.042058
3162,WA,6991,200993.506739
3163,WA,6992,187094.500729


### Checking for unique values of the columns
Mainly just looking out for inconsistencies in state and gender

In [15]:
from pyspark.sql import functions as F

In [16]:
customer_join_transaction.select("gender").distinct().show()

+-----------+
|     gender|
+-----------+
|Undisclosed|
|     Female|
|       Male|
+-----------+



In [17]:
customer_join_transaction.select("state").distinct().show()

+-----+
|state|
+-----+
|   NT|
|  ACT|
|   SA|
|  TAS|
|   WA|
|  QLD|
|  VIC|
|  NSW|
+-----+



## External dataset
- External dataset allows linking respective postcodes to their SA2 level index
- External dataset contains geometry for SA2 level which allows for geospatial analysis


In [23]:
import pandas as pd
import geopandas as gpd
import io
import requests

url = "https://www.matthewproctor.com/Content/postcodes/australian_postcodes.csv"
s = requests.get(url).content
df = pd.read_csv(io.StringIO(s.decode('utf-8')))
df.to_parquet("../data/curated/postcode_database.parquet")

In [24]:
df = spark.read.parquet("../data/curated/postcode_database.parquet")

Dataset contains alot of useless features that are not required for our analysis, hence we will be retaining only some of the important features.
- For instance, in this dataset, we have long, lat, long_precise, and lat_precise, we will retain the precise longitude and latitde as they are generated from Google Maps API 

In [25]:
# postcode_sdf = spark.createDataFrame(df[['postcode', 'SA2_MAINCODE_2016', 'Long_precise', 'Lat_precise']])
postcode_df = df.select(['postcode', 'SA2_MAINCODE_2016', 'Long_precise', 'Lat_precise']).toPandas()

In [26]:
postcode_df = (postcode_df.groupby(['postcode','SA2_MAINCODE_2016'], as_index=False)['Lat_precise','Long_precise'].mean())

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [27]:
revenue_by_region

,state,postcode,total_revenue_generated
0,ACT,200,101102.253779
1,ACT,2600,128671.340198
2,ACT,2601,143713.207764
3,ACT,2602,165889.186214
4,ACT,2603,163251.853817
...,...,...,...
3160,WA,6989,249449.669191
3161,WA,6990,169500.042058
3162,WA,6991,200993.506739
3163,WA,6992,187094.500729


In [28]:
revenue_by_region_1 = postcode_df.merge(revenue_by_region, how='inner')
revenue_by_region_1.count()

postcode                   5488
SA2_MAINCODE_2016          5488
Lat_precise                5488
Long_precise               5488
state                      5488
total_revenue_generated    5488
dtype: int64

In [29]:
revenue_by_region_1

,postcode,SA2_MAINCODE_2016,Lat_precise,Long_precise,state,total_revenue_generated
0,200,801051049.0,-35.277700,149.118764,ACT,101102.253779
1,800,701011002.0,-12.393279,130.776661,NT,268024.595375
2,801,701011002.0,-12.463440,130.845642,NT,215211.742051
3,804,701011007.0,-12.432480,130.846254,NT,171869.178297
4,810,701021010.0,-12.380000,130.873000,NT,262455.328626
...,...,...,...,...,...,...
5483,9013,305011105.0,-27.469771,153.025124,QLD,220583.835445
5484,9015,305011105.0,-27.469771,153.025124,QLD,176184.518459
5485,9464,302031038.0,-27.390000,153.066000,QLD,230439.000896
5486,9726,309101268.0,-28.016700,153.400000,QLD,158105.539360


In [31]:
gdf = gpd.GeoDataFrame(
      revenue_by_region_1, geometry=gpd.points_from_xy(revenue_by_region_1.Long_precise, revenue_by_region_1.Lat_precise))

In [57]:
gdf = gdf.drop(['Long_precise', 'Lat_precise'], axis=1)

In [32]:
gdf

,postcode,SA2_MAINCODE_2016,Lat_precise,Long_precise,state,total_revenue_generated,geometry
0,200,801051049.0,-35.277700,149.118764,ACT,101102.253779,POINT (149.11876 -35.27770)
1,800,701011002.0,-12.393279,130.776661,NT,268024.595375,POINT (130.77666 -12.39328)
2,801,701011002.0,-12.463440,130.845642,NT,215211.742051,POINT (130.84564 -12.46344)
3,804,701011007.0,-12.432480,130.846254,NT,171869.178297,POINT (130.84625 -12.43248)
4,810,701021010.0,-12.380000,130.873000,NT,262455.328626,POINT (130.87300 -12.38000)
...,...,...,...,...,...,...,...
5483,9013,305011105.0,-27.469771,153.025124,QLD,220583.835445,POINT (153.02512 -27.46977)
5484,9015,305011105.0,-27.469771,153.025124,QLD,176184.518459,POINT (153.02512 -27.46977)
5485,9464,302031038.0,-27.390000,153.066000,QLD,230439.000896,POINT (153.06600 -27.39000)
5486,9726,309101268.0,-28.016700,153.400000,QLD,158105.539360,POINT (153.40000 -28.01670)


In [33]:
rev_by_sa2 = gdf.groupby('SA2_MAINCODE_2016', as_index=False).agg({'total_revenue_generated':'sum'})

In [34]:
rev_by_sa2

,SA2_MAINCODE_2016,total_revenue_generated
0,101021007.0,4.246364e+05
1,101021008.0,2.515289e+05
2,101021009.0,2.515289e+05
3,101021010.0,2.515289e+05
4,101021011.0,1.305980e+06
...,...,...
2216,801111141.0,4.002018e+05
2217,901011001.0,1.642408e+05
2218,901021002.0,2.294536e+05
2219,901031003.0,2.543896e+05


In [35]:
# gdf.groupby('SA2_MAINCODE_2016', as_index=False)['Lat_precise','Long_precise'].mean()
rev_by_sa2 = (gdf.groupby('SA2_MAINCODE_2016', as_index=False).agg({'total_revenue_generated':'sum'})
    .merge(gdf.groupby('SA2_MAINCODE_2016', as_index=False)['Long_precise', 'Lat_precise'].mean()))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
gdf = gpd.GeoDataFrame(
      rev_by_sa2, geometry=gpd.points_from_xy(rev_by_sa2.Long_precise, rev_by_sa2.Lat_precise))

In [49]:
rev_by_sa2['SA2_MAINCODE_2016'] = rev_by_sa2['SA2_MAINCODE_2016'].astype(int).astype(str)

In [53]:
rev_by_sa2

,SA2_MAINCODE_2016,total_revenue_generated,Long_precise,Lat_precise,geometry
0,101021007,4.246364e+05,149.768193,-35.332661,POINT (149.76819 -35.33266)
1,101021008,2.515289e+05,149.233333,-35.366667,POINT (149.23333 -35.36667)
2,101021009,2.515289e+05,149.226329,-35.350386,POINT (149.22633 -35.35039)
3,101021010,2.515289e+05,149.253464,-35.348220,POINT (149.25346 -35.34822)
4,101021011,1.305980e+06,149.445979,-35.363561,POINT (149.44598 -35.36356)
...,...,...,...,...,...
2216,801111141,4.002018e+05,148.982743,-35.501929,POINT (148.98274 -35.50193)
2217,901011001,1.642408e+05,105.690449,-10.447525,POINT (105.69045 -10.44753)
2218,901021002,2.294536e+05,96.850173,-12.176433,POINT (96.85017 -12.17643)
2219,901031003,2.543896e+05,150.703069,-35.131759,POINT (150.70307 -35.13176)


In [54]:
rev_by_sa2.drop('geometry', inplace=True, axis=1)

In [59]:
geoJSON = gdf[['SA2_MAINCODE_2016','geometry']].to_json()

In [44]:
gdf['SA2_MAINCODE_2016'] = gdf['SA2_MAINCODE_2016'].astype(int).astype(str)

In [45]:
gdf

,SA2_MAINCODE_2016,total_revenue_generated,Long_precise,Lat_precise,geometry
0,101021007,4.246364e+05,149.768193,-35.332661,POINT (149.76819 -35.33266)
1,101021008,2.515289e+05,149.233333,-35.366667,POINT (149.23333 -35.36667)
2,101021009,2.515289e+05,149.226329,-35.350386,POINT (149.22633 -35.35039)
3,101021010,2.515289e+05,149.253464,-35.348220,POINT (149.25346 -35.34822)
4,101021011,1.305980e+06,149.445979,-35.363561,POINT (149.44598 -35.36356)
...,...,...,...,...,...
2216,801111141,4.002018e+05,148.982743,-35.501929,POINT (148.98274 -35.50193)
2217,901011001,1.642408e+05,105.690449,-10.447525,POINT (105.69045 -10.44753)
2218,901021002,2.294536e+05,96.850173,-12.176433,POINT (96.85017 -12.17643)
2219,901031003,2.543896e+05,150.703069,-35.131759,POINT (150.70307 -35.13176)


## Statistical Areas Level 2 - 2021 - Shapefile

In [39]:
sf = gpd.read_file("../data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")

In [40]:
sf = sf[['SA2_CODE21','geometry']]

In [55]:
gdf = sf.merge(rev_by_sa2, right_on='SA2_MAINCODE_2016', left_on='SA2_CODE21')

In [56]:
gdf

,SA2_CODE21,geometry,SA2_MAINCODE_2016,total_revenue_generated,Long_precise,Lat_precise
0,101021007,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4...",101021007,424636.426495,149.768193,-35.332661
1,101021008,"POLYGON ((149.21899 -35.36738, 149.21800 -35.3...",101021008,251528.885714,149.233333,-35.366667
2,101021009,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3...",101021009,251528.885714,149.226329,-35.350386
3,101021010,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3...",101021010,251528.885714,149.253464,-35.348220
4,101021012,"POLYGON ((149.19572 -35.36126, 149.19970 -35.3...",101021012,468763.039302,149.203913,-35.370800
...,...,...,...,...,...,...
2082,801111141,"POLYGON ((148.80407 -35.37619, 148.80417 -35.3...",801111141,400201.789443,148.982743,-35.501929
2083,901011001,"POLYGON ((105.67393 -10.41566, 105.67399 -10.4...",901011001,164240.839186,105.690449,-10.447525
2084,901021002,"MULTIPOLYGON (((96.91512 -12.14044, 96.91513 -...",901021002,229453.592303,96.850173,-12.176433
2085,901031003,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556...",901031003,254389.609736,150.703069,-35.131759


In [57]:
gdf.describe()

,total_revenue_generated,Long_precise,Lat_precise
count,2.087000e+03,2087.000000,2087.000000
mean,4.690999e+05,143.866746,-31.915911
std,4.935818e+05,11.578704,6.294713
min,2.184746e+04,37.196119,-45.901538
25%,1.860041e+05,143.189520,-35.442000
50%,2.821303e+05,147.361875,-33.739860
75%,5.487173e+05,151.222047,-27.682353
max,5.911230e+06,167.952586,-5.447754


In [ ]:
import folium
m = folium.Map(location=[25, 133], 
               tiles="cartodb positron",
               zoom_start=3,
               zoom_control=False,
               width=475,
               height=500,
               html='<div style="font-size: 10pt"</div>')

title = '''
        <h3 align="left" style="font-size:16px"><b>
        Total Dollar Spent by SA2 Code</b></h3>'''.format('Corpus Christi')

m.get_root().html.add_child(folium.Element(title))

# refer to the folium documentations on more information on how to plot aggregated data.
c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=gdf, # data source
    columns=['SA2_MAINCODE_2016','total_revenue_generated'], # the columns required
    key_on='properties.SA2_MAINCODE_2016', # this is from the geoJSON's properties
    bins=4,
    fill_color='YlGn', # color scheme
    line_opacity=0.3,
    nan_fill_color='white',
    legend_name='Total Spending by SA2 from August 2021 to February 2022'
)

c.add_to(m)

m